## Modelling Notebook

**Layer**: Silver

**Domain**: Risk-free

**Action**: Modelling of the zero coupon bond forward rates using observable market data

In [0]:
import pandas as pd
import numpy as np

## Section 1 - Load Inputs

In [0]:
# Set as a parameter here for now
as_of_date = '2025-06-30'

In [0]:
# Read cleaned and enriched inputs from silver layer
df_yields = spark.table("workspace.riskfree_silver.003_rbnz_yields_transformed") \
    .filter(f"(date = '{as_of_date}' AND series = 'Official Cash Rate (OCR)' AND group = 'Cash rate') OR (date = '{as_of_date}' AND series NOT LIKE '%year%' AND group = 'Secondary market government bond yields')")
df_amounts   = spark.table("workspace.riskfree_silver.002_nzdm_govtbonds_onissue_enriched").filter(f"as_of_date = '{as_of_date}' AND group = 'Secondary market government bond yields'")

In [0]:
display(df_yields)
display(df_amounts)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

## Section 2 - Pre-processing of data

In [0]:
# Convert spark to pandas for modelling
pdf_yields = df_yields.toPandas()
pdf_amounts = df_amounts.toPandas()

# Merge the dataframes based on series_id
merged_pdf = pdf_yields.merge(pdf_amounts, on='series_id', how='left')

## Section 3 - Bootstrapping of zero-coupon forward rates

The one-month forward rate is determined from the one-month Treasury bill.

Nominal Government bonds are decomposed into maturity and individual coupon payments to produce a set of equivalent zero-coupon nominal bonds maturing on the 15th of the month

A forward rate is determined for the shortest nominal Government bond, for the period up until the first nominal bond matures. For the period between the first nominal bond and the nominal second bond a forward rate is determined so that the second nominal bond market value is equalled using the previous forward rates as well. This process is repeated to solve for each successive forward rate until all nominal bonds have been valued.

In [0]:
# Sort by term_yr and keep only values needed
df = merged_pdf.sort_values(by='term_yr')[["term_yr", "yield_decimal"]].rename(columns={"yield_decimal": "spot_rate_pa"})

# Discount Factors from spot rates
df["discount_factor"] = 1 / (1 + df["spot_rate_pa"]) ** df["term_yr"]

# Bootstrap forward rates between adjacent terms
fwd_rates = []
for i in range(1, len(df)):
    t0 = df.loc[i-1, "term_yr"]
    t1 = df.loc[i, "term_yr"]
    D0 = df.loc[i-1, "discount_factor"]
    D1 = df.loc[i, "discount_factor"]
    fwd = (D0 / D1) ** (1 / (t1 - t0)) - 1
    fwd_rates.append((t0, t1, fwd))

# Final forward rate DataFrame
df_fwd = pd.DataFrame(fwd_rates, columns=["from_year", "term_yr", "fwd_rate_pa"])[["term_yr", "fwd_rate_pa"]]
df = df.merge(df_fwd, on="term_yr", how="left")[["term_yr", "spot_rate_pa", "fwd_rate_pa"]].fillna({"fwd_rate_pa": df["spot_rate_pa"]})

# Display result
display(df)

Databricks visualization. Run in Databricks to view.